Exercise:  09-2 Ingress
-------------------

![](demo/images/Microservices-REST.png)

Source: Book Microservices Recipes
- - -

The example consists of three microservices: **Order**, **Customer** and **Catalog**.

**Order** uses **Catalog** and **Customer** with REST interface. In addition, each microservice offers some HTML pages.

Instead of the Apache web server, which is configured as [Reverse Proxy](https://github.com/ewolff/microservice-kubernetes/blob/master/microservice-kubernetes-demo/apache/000-default.conf), the Kubernetes resource Ingress is used.

In [ ]:
%%bash
kubectl create namespace ms-rest
# ! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml (obsolet!)
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml

After starting the services, we create the Ingress resources:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /\$2
  name: order
  namespace: ms-rest 
  labels:
    app: order
spec:
  rules:
  - http:
      paths:
      - path: /order/
        pathType: Prefix
        backend:
          service:
            name: order
            port:
              number: 8080
---
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: catalog
  namespace: ms-rest
  labels:
    app: catalog
spec:
  rules:
  - http:
      paths:
      - path: /catalog
        pathType: Prefix
        backend:
          service:
            name: catalog
            port:
              number: 8080
---
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: customer 
  namespace: ms-rest
  labels:
    app: customer 
spec:
  rules:
  - http:
      paths:
      - path: /customer
        pathType: Prefix
        backend:
          service:
            name: customer
            port:
              number: 8080  
%EOF%


Checking the created resources

In [ ]:
! kubectl get all,ingress -n ms-rest

All microservices can now be reached using `https` and the same DNS or IP address.

In [ ]:
%%bash
export SERVER=https://$(cat ~/work/server-ip)
echo "Kunden    : ${SERVER}/customer"
echo "Produkte  : ${SERVER}/catalog"
echo "Bestellung: ${SERVER}/order/index.html"

***
### Ingress Service (nginx Server)

In the current environment, an nginx server handles the ingress functionality. This server runs as pods in the ingress-nginx namespace.

We can output the configuration file from the nginx server:

In [ ]:
! kubectl exec daemonset/nginx-ingress-microk8s-controller -n ingress -- cat /etc/nginx/nginx.conf | grep location

For testing, the `kubectl apply -f -` which creates the ingress resources can be replaced with `kubectl delete -f -` and then the above command can be run again.

Then the `location` entries for `customer`, `catalog` and `order` should no longer exist.

- - -

Clean up

In [ ]:
! kubectl delete namespace ms-rest

***
Questions
======

Answer the questions individually or in groups and compare them with the answers


What an ingress?
<details><summary>Answer</summary><p>
An API object that manages external access to the services in a cluster, typically using HTTP.
The ingress service roughly corresponds to the reverse proxy pattern.
</p></details>

---

What kind of network structure does Kubernetes use?
<details><summary>Answer</summary><p>
Unlike Docker, Kubernetes uses a flat network structure.
* Each container can communicate with each other without NAT.
* All Kubernetes nodes can communicate with all containers (and in the other direction) without NAT.
* The IP that a container sees of itself is also the IP that every other node or container on the network sees of it.
</p></details>

---